In [ ]:
!pip install efficientnet_pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms
from torchsummary import summary
from efficientnet_pytorch import EfficientNet
from torch.utils.tensorboard import SummaryWriter

import os
import time
import copy

## 하이퍼파라미터 세팅

In [ ]:
input_size = (3, 128, 128)
batch_size = 64
test_batch_size = 1000
epochs = 10
lr = 0.001
momentum = 0.5
no_cuda = True
seed = 555
log_interval = 200
use_cuda = not no_cuda and torch.cuda.is_available()

torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")
writer = SummaryWriter('./runs/experiment')
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
print("Set vars and Device done")

## 데이터 전처리

In [ ]:
data_transform = {
    'train': transforms.Compose([
                    transforms.Resize(200, 200)
                    , transforms.RandomRotation(30)
                    , transforms.RandomHorizontalFlip()
                    , transforms.ToTensor()
                    , transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
                transforms.ToTensor()
                , transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                , transforms.Resize(200,200)
    ])
}

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST( '/FashionMNIST' , train=True, download=True , transform=data_transform['train'] )
    , batch_size = batch_size
    , shuffle=True
    , **kwargs
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/FashionMNIST', train=False, download=True, transform=data_transform['val'])
    , batch_size=test_batch_size
    , shuffle=True
    , **kwargs
)

## 데이터 시각화

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

## 모델 불러오기

In [ ]:
model = EfficientNet.from_pretrained('efficientnet-b3', num_classes=10)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

since = time.time()
best_model_weights = copy.deepcopy(model.state_dict())
best_acc = 0.0

## 모델 학습

In [ ]:
for epoch in range(epochs):
    print('Epoch {}/{}'.format(epoch, epochs-1))
    print('-' * 10)

    for phase in ['train', 'eval']:
        if phase == 'train': model.train()
        else: model.eval()

    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in [train_loader, test_loader]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        with torch.set_grad_enabled(phase == 'train'):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            if phase == 'train':
                loss.backward()
                optimizer.step()

    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)

    if phase == 'train': scheduler.step()

    epoch_loss = running_loss / dataset_sizes[phase]
    epoch_acc = running_corrects.double() / dataset_sizes[phase]

    print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

    if phase == 'val' and epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_weights = copy.deepcopy(model.state_dict())